# DATA COLLECTION

In [ ]:
pip install odfpy

In [2]:
import pandas as pd
from io import BytesIO
import numpy as np
import os
from google.colab import drive

Recupero i dataset dal progetto github

In [3]:
!git clone https://github.com/morkioj/parco_veicoli.git

Cloning into 'parco_veicoli'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 35 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 14.55 MiB | 15.40 MiB/s, done.
Resolving deltas: 100% (8/8), done.


**Importazione files ods**
* Per ogni file Circolante_FTS_Autovetture_AAAA.ods presente nella cartella circolante_fts_autovetture estraggo il contenuto del foglio "Nazionale" e da questo genero un dataframe pandas
* A questo df, aggiungo la colonna 'Anno' estrando il valore dal nome del file
* Inserisco il df in una lista di df
* Iterati tutti i file nella cartella, unisco tutti i df presenti nella lista in un nuovo dataframe (df_orig)

In [ ]:
#drive.mount('/content/drive')
#folder_path="/content/drive/MyDrive/Colab Notebooks/dataset/fts/"
folder_path="/content/parco_veicoli/dataset/fabbrica_tipo_serie_circolante/"
sheet_name="Nazionale"
prefisso_file="Circolante_FTS_Autovetture"
text_to_search= ["hybrid"]
vetture_per_anno=10
file_consumi="ibride_consumi_litri-100km.csv"
file_nazione="fabbrica_nazione_continente.csv"

file_path=""
df_list=[]

#Lista dei file presenti nella cartella
file_list = os.listdir(folder_path)

for file_name in file_list:
  file_path = os.path.join(folder_path, file_name)
  if(file_name.split('_20')[0]==prefisso_file and os.path.splitext(file_name)[1] == ".ods" ):
    with open(file_path, 'rb') as f:
      file_content = f.read()
    file_stream = BytesIO(file_content)
    #creo il dataframe tramite lo stream di lettura del file (intestazione colonne indice 3)
    df_file =pd.read_excel(io=file_stream,sheet_name=sheet_name,engine='odf',header=3)
    #aggiungo una colonna al ds per rappresentare l'anno
    df_file['Anno'] = int(file_name.split('_')[-1].replace('.ods',''))
    #aggiungo il df alla lista
    df_list.append(df_file)
    print(f"{file_name} importato...")
  else:
    print(f"{file_name} non importato...")
df_orig=pd.concat(df_list,ignore_index=True)

# DATA CLEANING

1. Rimuvo le righe che rappresentano i totali nazionali
2. Sostituisco i valori nan delle colonne testuali (Fabbrica, Tipo, Serie) con stringa vuota
3. Sostituisco i valori nan delle colonne numeriche (Totale) con 0

In [ ]:
#1
df_orig = df_orig[df_orig['Fabbrica']!='TOTALE NAZIONALE']
#2
df_orig[['Fabbrica','Tipo','Serie']] =df_orig[['Fabbrica','Tipo','Serie']].fillna('')
#3
df_orig[['Totale']] =df_orig[['Totale']].fillna(0)

# DATA TRASFORMATION

**CALCOLO DEL RISPARMIO MEDIO DERIVANTE DALL'IBRIDO**

STEP I: ricerca modelli maggiormente diffusi
1. Estraggo le sole vetture i cui motore è ibrido (filtro la serie con delle parolo chiave
2. Per ogni modello calcolo il numero in circolazione annuale (raggruppo per modello ed anno e sommo il totale)
3. Prendo i primo 10 modelli piu diffusi annualmente (ordino per anno crescente e totale decrescente e prendo i primi 10)


In [ ]:
#1
df_wrk=df_orig[df_orig['Serie'].str.contains('|'.join(text_to_search),case=False,na=False)]
#2
df_grouped=df_wrk.groupby(['Fabbrica','Tipo','Anno'])['Totale'].sum().reset_index()
#3
df_grouped=df_grouped.sort_values(['Anno','Totale'], ascending=[True,False]).groupby('Anno').head(vetture_per_anno)

STEP II: Associazione dei consumi ai modelli
4. Ricerca dei consumi (solo endotermico e ibrido) dei dieci modelli piu diffusi
5. Inserisco i dati trovati all'interno di un csv che verrà poi importato ed associato ai modelli

In [ ]:
#4
file_consumi="ibride_consumi_litri-100km.csv"
file_path = os.path.join(folder_path, file_consumi)
df_consumi=pd.read_csv(file_path,sep=';')
#5
df_export_consumi=pd.merge(df_grouped,df_consumi,on=['Fabbrica','Tipo']).reset_index(drop=True)

STEP III: Calcolo del risparmio medio
6. Calcolo una media ponderata per ogni anno dei consumi e quindi il risparmio tra ibrido e benzina


In [ ]:
#6
consumi_medi = df_export_consumi.groupby('Anno').apply(lambda x: pd.Series({
  'Consumo_Std_Medio': (x['Consumo_Std'] * x['Totale']).sum() / x['Totale'].sum(),
  'Consumo_Hyb_Medio': (x['Consumo_Hyb'] * x['Totale']).sum() / x['Totale'].sum()
})).reset_index()

consumi_medi['Risparmio']=(consumi_medi['Consumo_Std_Medio'] - consumi_medi['Consumo_Hyb_Medio']) / consumi_medi['Consumo_Std_Medio'] * 100

Elenco delle case automobilistiche per numero di vetture annuo
Associazione della nazione e del continente alla casa automobilistica

**CASE AUTOMOBILISTICHE**

1. Estrazione delle singole case automobilistiche
2. All'elenco dei singoli modelli in circolazione, associo il continente e la nazione della fabbrica



In [ ]:
df_fab_vet_ann=df_orig.groupby(['Fabbrica','Anno'])['Totale'].sum().reset_index()
file_path = os.path.join(folder_path, file_nazione)
df_nazione=pd.read_csv(file_path,sep=';')
df_fabbrica_nazione=pd.merge(df_fab_vet_ann,df_nazione,on=['Fabbrica']).reset_index(drop=True)

**ELENCO DISTINTO DELLE CASE AUTOMOBILISTICHE**

In [ ]:
s=pd.Series(df_fab_vet_ann['Fabbrica'].unique())